In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pymongo
from pprint import pprint

In [2]:
client = pymongo.MongoClient()
db = client.metrics

In [3]:
def group_by(df, bycols, agg_map):
    """

    @param df:      DataFrame
    @param bycols:  str or list
                        Column(s) to group by
    @param agg_map: dictionary or list of 2-tuples
                        Mapping from column to aggregate function e.g. [("city", "count"), ("salary", "mean"]
    @return:        DataFrame
                        Flattened dataframe, with multi-level index removed
    """
    grps = []
    if type(bycols) == str:
        bycols = [bycols]

    if type(agg_map) == dict:
        agg_map = agg_map.items()

    for k,v in agg_map:
        grp = df[bycols + [k]].groupby(bycols, ).agg(v)
        grp.reset_index(inplace=True)
        grp["%s(%s)" % (v,k)] = grp[k]
        del grp[k]
        grps.append(grp)

    m = grps[0]
    for grp in grps[1:]:
        m = pd.merge(m, grp, on=bycols, how="inner")
    return m

In [4]:
from bson.son import SON # needed to ensure dictionary is ordered (python default is not)
import hashlib

def hash_feats(fts):
    vals = fts.values
    joined = "|".join(map(str,vals))
    return hashlib.sha224(joined).hexdigest()

def get_df_sorted_by_f1score(collection, params=None, filter_cols=True):
    if not params:
        params = []
    if type(params) == str:
        params = params.split(",")
    
    project = {
            "weighted_f1_score":"$WEIGHTED_MEAN_CONCEPT_CODES.f1_score",
            "micro_f1_score":  "$MICRO_F1.f1_score",
            "micro_recall":    "$MICRO_F1.recall",
            "micro_precision": "$MICRO_F1.precision",
    
    # PARAMETERS            
            "window_size":    "$parameters.window_size",
            "feats":          "$parameters.extractors",
            "count": {        "$size" : "$parameters.extractors" },
            "asof" :          "$asof",
            "_id":1
    }
    
    # No count for HMM
    if "_hmm" in collection.lower():
        del project["count"]
    
    for param in params:
        project[param] = "$parameters." + param

    feats_pipeline = [{
        "$project": project
    },
    {
        "$match":{
            "micro_f1_score": { "$exists" : True }        
        }
    },
    {
        "$sort":{
            "micro_f1_score": -1
        }
    },
    ]
    
    rows = [row for row in db[collection].aggregate(feats_pipeline)]
    df = pd.DataFrame(rows).sort_values("micro_f1_score", ascending=False)
    if params:
        df["hs_params"] = df[params].apply(hash_feats, axis=1)
        
    if filter_cols:
        cols = ["micro_f1_score", "micro_recall" ,"micro_precision" ] + params
        return df[cols]
    return df

In [5]:
def get_window_classifier_results(prefix):
    collections = "WINDOW_CLASSIFIER_BR,WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS,WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS".split(",")
    dfs = []
    for c in collections:
        col = prefix + c
        print col
        df = dict(get_df_sorted_by_f1score(col).iloc[0,:])
        df["Collection_" + prefix[:-1]] = col.replace(prefix,"")
        dfs.append(df)
    return pd.DataFrame(dfs).sort_values("micro_f1_score", ascending=False)

# Which Problem Transformation Method Was Best?

In [6]:
def round_data(df, places=3):
    df_copy = df.copy()
    fmt_str = "{0:." + str(places) + "f}"
    cols = set([v for v in df_copy.columns.values if "micro_" in v])
    for c in cols:
        df_copy[c] = df[c].apply(lambda d: fmt_str.format(d))  
    return df_copy

## Coral Bleaching

In [7]:
df = get_window_classifier_results("CB_TAGGING_VD_")
df = round_data(df, 4)
df["Collection_CB_TAGGING_VD,micro_f1_score,micro_recall,micro_precision".split(",")]

CB_TAGGING_VD_WINDOW_CLASSIFIER_BR
CB_TAGGING_VD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS
CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS


,Collection_CB_TAGGING_VD,micro_f1_score,micro_recall,micro_precision
2,WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS,0.8317,0.7863,0.8826
1,WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS,0.8316,0.7861,0.8827
0,WINDOW_CLASSIFIER_BR,0.8247,0.7708,0.8866


## Skin Cancer

In [8]:
df = get_window_classifier_results("SC_TAGGING_VD_")
for c in ["micro_f1_score","micro_recall","micro_precision"]:
    df[c] = df[c].apply(lambda d: "{0:.4f}".format(d))  
df["Collection_SC_TAGGING_VD,micro_f1_score,micro_recall,micro_precision".split(",")]

SC_TAGGING_VD_WINDOW_CLASSIFIER_BR
SC_TAGGING_VD_WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS
SC_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS


,Collection_SC_TAGGING_VD,micro_f1_score,micro_recall,micro_precision
1,WINDOW_CLASSIFIER_LBL_POWERSET_MULTICLASS,0.8087,0.7749,0.8456
2,WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS,0.8087,0.7749,0.8456
0,WINDOW_CLASSIFIER_BR,0.8011,0.7592,0.8478


** Unsurprisingly in this case, as there were only two MLC labels, the score for LBL powerset and Common tag are the same **

** HOWEVER - why is the multiclass version that much better? It does OVR, and with only 2 records difference, this makes no sense to me **

# Hyper Parameter Tuning Results

In [9]:
# Rows to print - df.head
ROWS = 5

## Window Based Classifier - Hyper Parameter Tuning

### Coral Bleaching

In [10]:
params = "dual,C,penalty,fit_intercept,multi_class,window_size".split(",")
collection = "CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS_HYPER_PARAM_TUNING"

df = get_df_sorted_by_f1score(collection, params)

round_data(df.head(ROWS),4)
#df

,micro_f1_score,micro_recall,micro_precision,dual,C,penalty,fit_intercept,multi_class,window_size
0,0.8319,0.7814,0.8892,True,0.5,l2,True,ovr,9
1,0.8318,0.7814,0.8892,False,0.5,l2,True,ovr,9
2,0.8317,0.7863,0.8826,True,1.0,l2,True,ovr,9
3,0.8317,0.7863,0.8826,False,1.0,l2,True,ovr,9
4,0.8286,0.7863,0.8756,False,1.0,l1,True,ovr,9


### Skin Cancer

In [11]:
params = "dual,C,penalty,fit_intercept,multi_class,window_size".split(",")
collection = "SC_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS_HYPER_PARAM_TUNING"

df = get_df_sorted_by_f1score(collection, params)
#df = df[df.dual==True]
#df = df[df.C==0.5]
#df = df[df.penalty =='l2']
round_data(df.head(ROWS),3)

,micro_f1_score,micro_recall,micro_precision,dual,C,penalty,fit_intercept,multi_class,window_size
0,0.810,0.773,0.850,True,0.5,l2,True,ovr,9
1,0.810,0.772,0.850,False,0.5,l2,True,ovr,9
2,0.809,0.775,0.846,False,1.0,l2,True,ovr,9
3,0.809,0.775,0.846,True,1.0,l2,True,ovr,9
4,0.809,0.777,0.843,False,1.0,l1,True,ovr,9


### <span style="color:red">Optimal Hyper Parameters are the same for both datasets </span>

## CRF Performance - Hyper Parameter Tuning 

### Coral Bleaching

In [12]:
cols = "micro_f1_score,micro_recall,micro_precision,feature_possible_states,feature_possible_transitions,c2,window_size".split(",")
df = get_df_sorted_by_f1score("CB_TAGGING_VD_CRF_MOST_COMMON_TAG_HYPERPARAM_OPT",
                         "feature_possible_states,feature_possible_transitions,c2,window_size".split(","))

#df = df[df.c2==0.1]
#df = df[df.feature_possible_transitions==True]

round_data(df[cols].head(ROWS), 3)

,micro_f1_score,micro_recall,micro_precision,feature_possible_states,feature_possible_transitions,c2,window_size
0,0.831,0.781,0.888,False,True,0.1,9
1,0.831,0.782,0.888,False,False,0.1,9
2,0.831,0.775,0.896,False,False,1.0,9
3,0.830,0.775,0.895,False,True,1.0,9
4,0.813,0.743,0.897,False,True,10.0,9


### Skin Cancer

In [13]:
cols = "micro_f1_score,micro_recall,micro_precision,feature_possible_states,feature_possible_transitions,c2,window_size".split(",")
df = get_df_sorted_by_f1score("SC_TAGGING_VD_CRF_MOST_COMMON_TAG_HYPERPARAM_OPT",
                         "feature_possible_states,feature_possible_transitions,c2,window_size".split(","))

#df = df[df.c2==1.0]
#df = df[df.feature_possible_transitions==True]
round_data(df[cols].head(ROWS), 3)

,micro_f1_score,micro_recall,micro_precision,feature_possible_states,feature_possible_transitions,c2,window_size
0,0.799,0.758,0.846,False,True,1.0,9
1,0.798,0.756,0.844,False,False,1.0,9
2,0.789,0.754,0.827,False,False,0.1,9
3,0.789,0.754,0.827,False,True,0.1,9
4,0.785,0.723,0.859,False,True,10.0,9


### <span style="color:red">Optimal C2 differs between datasets</span>

## HMM - Hyper Parameter Tuning (Features in this case)

### Coral Bleaching

In [14]:
params = "extractors".split(",")
collection = "CB_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS"

df = get_df_sorted_by_f1score(collection, params)
round_data(df.head(ROWS))

,micro_f1_score,micro_recall,micro_precision,extractors
0,0.773,0.797,0.750,stemmed_unigrams
1,0.769,0.785,0.754,unigrams


### Skin Cancer

In [15]:
params = "extractors".split(",")
collection = "SC_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS"

df = get_df_sorted_by_f1score(collection, params)
round_data(df.head(ROWS))

,micro_f1_score,micro_recall,micro_precision,extractors
0,0.674,0.733,0.625,stemmed_unigrams
1,0.672,0.719,0.631,unigrams


## Average Perceptron - Hyper Parameter Tuning Results

### Coral Bleaching

In [16]:
model = "AVG_PERCEPTRON_MOST_COMMON_TAG_HYPER_PARAM_TUNING_NEW"
df = get_df_sorted_by_f1score("CB_TAGGING_VD_" + model, 
                              "average_weights,tag_history,window_size")
df.head(ROWS)["micro_f1_score,micro_recall,micro_precision,average_weights,tag_history,window_size,window_size".split(",")]

,micro_f1_score,micro_recall,micro_precision,average_weights,tag_history,window_size,window_size
0,0.829924,0.778857,0.888157,True,2,9,9
1,0.829481,0.778466,0.887651,True,1,9,9
2,0.828734,0.776121,0.889000,True,3,9,9
3,0.821849,0.767943,0.883894,True,0,9,9
4,0.784722,0.810157,0.760836,False,1,9,9


### Skin Cancer

In [25]:
model = "AVG_PERCEPTRON_MOST_COMMON_TAG_HYPER_PARAM_TUNING_NEW"
df = get_df_sorted_by_f1score("SC_TAGGING_VD_" + model, 
                              "average_weights,tag_history,window_size")
df.head(ROWS)["micro_f1_score,micro_recall,micro_precision,average_weights,tag_history,window_size,window_size".split(",")]

,micro_f1_score,micro_recall,micro_precision,average_weights,tag_history,window_size,window_size
0,0.808588,0.767088,0.854835,True,1,9,9
1,0.802696,0.755411,0.856295,True,0,9,9
2,0.749183,0.734237,0.764749,False,1,9,9
3,0.703646,0.701285,0.706023,False,0,9,9


### <span style="color:red">Optimal tag_history and tag_ngram_size differ between datasets</span>

# Test Set Performance

## Window Based Classifier

### Coral Bleaching

In [18]:
params = "dual,C,penalty,fit_intercept,multi_class,window_size".split(",")
collection = "TEST_CB_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS"

df = get_df_sorted_by_f1score(collection, params)
round_data(df.head(ROWS),3)

,micro_f1_score,micro_recall,micro_precision,dual,C,penalty,fit_intercept,multi_class,window_size
0,0.842,0.802,0.885,True,0.5,l2,True,ovr,9


### Skin Cancer

In [19]:
params = "dual,C,penalty,fit_intercept,multi_class,window_size".split(",")
collection = "TEST_SC_TAGGING_VD_WINDOW_CLASSIFIER_MOST_COMMON_TAG_MULTICLASS"

df = get_df_sorted_by_f1score(collection, params)
round_data(df.head(ROWS),3)

,micro_f1_score,micro_recall,micro_precision,dual,C,penalty,fit_intercept,multi_class,window_size
0,0.814,0.779,0.853,True,0.5,l2,True,ovr,9


## CRF

### Coral Bleaching

In [20]:
cols = "micro_f1_score,micro_recall,micro_precision,feature_possible_states,feature_possible_transitions,c2,window_size".split(",")
df = get_df_sorted_by_f1score("TEST_CB_TAGGING_VD_CRF_MOST_COMMON_TAG",
                         "feature_possible_states,feature_possible_transitions,c2,window_size".split(","))
round_data(df[cols].head(ROWS),3)

,micro_f1_score,micro_recall,micro_precision,feature_possible_states,feature_possible_transitions,c2,window_size
0,0.840,0.803,0.881,False,True,0.1,9


## Skin Cancer

In [21]:
cols = "micro_f1_score,micro_recall,micro_precision,feature_possible_states,feature_possible_transitions,c2,window_size".split(",")
df = get_df_sorted_by_f1score("TEST_SC_TAGGING_VD_CRF_MOST_COMMON_TAG",
                         "feature_possible_states,feature_possible_transitions,c2,window_size".split(","))
round_data(df[cols].head(ROWS),3)

,micro_f1_score,micro_recall,micro_precision,feature_possible_states,feature_possible_transitions,c2,window_size
0,0.804,0.759,0.855,False,True,1.0,9


## HMM

### Coral Bleaching

In [22]:
params = "extractors".split(",")
collection = "TEST_CB_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS"

df = get_df_sorted_by_f1score(collection, params)
round_data(df.head(ROWS))

,micro_f1_score,micro_recall,micro_precision,extractors
0,0.764,0.803,0.728,stemmed_unigrams


## Skin Cancer

In [23]:
params = "extractors".split(",")
collection = "TEST_SC_TAGGING_VD_HMM_MOST_COMMON_TAG_MULTICLASS"

df = get_df_sorted_by_f1score(collection, params)
round_data(df.head(ROWS))

,micro_f1_score,micro_recall,micro_precision,extractors
0,0.675,0.731,0.628,stemmed_unigrams


## Averaged Perceptron

### Coral Bleaching

In [24]:
model = "AVG_PERCEPTRON_MOST_COMMON_TAG"
df = get_df_sorted_by_f1score("TEST_CB_TAGGING_VD_" + model, 
                              "tag_history,tag_plus_word,tag_ngram_size,window_size")
df.head(ROWS)["micro_f1_score,micro_recall,micro_precision,tag_history,tag_plus_word,tag_ngram_size,window_size".split(",")]

KeyError: 'micro_f1_score'

### Skin Cancer

In [ ]:
model = "AVG_PERCEPTRON_MOST_COMMON_TAG"
df = get_df_sorted_by_f1score("TEST_SC_TAGGING_VD_" + model, 
                              "tag_history,tag_plus_word,tag_ngram_size,window_size")
df.head(ROWS)["micro_f1_score,micro_recall,micro_precision,tag_history,tag_plus_word,tag_ngram_size,window_size".split(",")]

### Get Train, Test Scores